# Visualize geospatial data - Folium

In [2]:
import os
from pathlib import Path
import pandas as pd
from pyzipcode import ZipCodeDatabase
import matplotlib.pyplot as plt



## 1. Convert zipcode to state, longitude, latitude

In [3]:
zcdb = ZipCodeDatabase()

data1 = pd.read_csv('./DataSet1__24Apr2018.txt', sep=" ")

df=data1[data1['CommunityZipCode'].notnull()]
df.CommunityZipCode = df.CommunityZipCode.astype(int)
df=df[df['CommunityZipCode']>10000]


df['state']= df.apply(lambda x : zcdb[x.CommunityZipCode].state, axis=1)
df['x']= df.apply(lambda x : zcdb[x.CommunityZipCode].longitude, axis=1)
df['y']= df.apply(lambda x : zcdb[x.CommunityZipCode].latitude, axis=1)


new_df=df.iloc[:,24:28]



/Users/YiChen/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [4]:
new_df.head()

,state,x,y
0,WI,-87.9267,44.4914
1,IL,-88.1383,42.1614
2,GA,-83.8968,33.8769
3,FL,-80.0794,26.5747
4,FL,-80.1891,26.0218


# Folium

### Python Data. Leaflet.js Maps.
### Folium builds on the data wrangling strengths of the Python ecosystem and the mapping strengths of the Leaflet.js library. Manipulate your data in Python, then visualize it in on a Leaflet map via Folium.


## 2. Use Folium tovisualize which area has the largest data
 
### (1) FastMarkerCluster

In [5]:
import folium
from folium.plugins import FastMarkerCluster
test=new_df[:70000]

m = folium.Map(location=[new_df['y'].mean(),new_df['x'].mean()], zoom_start=4)
#Initialize Map
FastMarkerCluster(test[['y','x']].values.tolist()).add_to(m)
#Add marker to Map
m

# over 70000 data can't show map on jupyter notebook, however, map can check as html format
#m.save("Cluster.html")



In [6]:
count=new_df.groupby(by='state').count().x
Group_df=new_df.groupby(by='state').mean()
Group_df['count']=count
Group_df=Group_df.reset_index()
Group_df.sort_values(by='count',ascending=False).head()

,state,x,y,count
4,FL,-80.901905,26.898227,26533
6,IL,-87.926218,41.858231,23834
20,TX,-96.617079,30.327623,17176
0,AZ,-112.238810,33.564235,14722
15,OH,-84.011239,39.410282,14131


### (2)choropleth 

In [7]:


import json

us_states = os.path.join( 'us-states.json')
geo_json_data = json.load(open(us_states))

m = folium.Map([43, -100], zoom_start=4)
m.choropleth(
    geo_data=geo_json_data,
    data=Group_df,
    columns=['state', 'count'],
    threshold_scale=[0, 5000, 10000, 16000, 22000, 27000],
    key_on='feature.id',
    fill_color='YlOrBr',
    legend_name='Senior Housing client',
    highlight=True
)
#check http://colorbrewer2.org/#type=sequential&scheme=YlOrRd&n=3 to change fill_color


m